Problem 4

In [3]:
import numpy as np
import matplotlib.pyplot as plt


np.seterr(invalid='ignore', over='ignore')  # suppress warning caused by division by inf

def f(x):
    return 1/(1 + np.exp(3*(x-3))) * 10 * x**2  + 1 / (1 + np.exp(-3*(x-3))) * (0.5*(x-10)**2 + 50)

def fprime(x):
    return 1 / (1 + np.exp((-3)*(x-3))) * (x-10) + 1/(1 + np.exp(3*(x-3))) * 20 * x + (3* np.exp(9))/(np.exp(9-1.5*x) + np.exp(1.5*x))**2 * ((0.5*(x-10)**2 + 50) - 10 * x**2) 

# x = np.linspace(-5,20,100)
# plt.plot(x,f(x), 'k')
# plt.show()

def gd(x, lr):
    curr = x
    for _ in range(10000):
        prev = curr
        curr = step(prev, lr)
    return curr

def step(x, lr):
    return x - lr * fprime(x)


def find_minima(lr):
    x_converge = []
    start = np.random.uniform(-5, 20, 5)
    for x in start:
        x_converge = x_converge + [gd(x, lr)]
    return x_converge

In [4]:
np.random.seed(9876)
print(find_minima(0.01))

[-0.001780002229631524, 10.000002009367371, 10.000002009367371, 10.000002009367371, 10.000002009367195]


In [5]:
np.random.seed(5432)
print(find_minima(0.3))

[10.00000200936728, 10.00000200936728, 10.000002009367284, 10.000002009367284, 10.00000200936728]


In [6]:
np.random.seed(10)
print(find_minima(4))

[nan, nan, nan, nan, nan]


Problem 5

In [7]:
import numpy as np

class Convolution1d :
    def __init__(self, filt) :
        self.__filt = filt
        self.__r = filt.size
        self.T = TransposedConvolution1d(self.__filt)

    def __matmul__(self, vector) :
        r, n = self.__r, vector.size

        return np.asarray([self.__filt.dot(vector[i:i+r]) for i in np.arange(n-r+1)])
    
class TransposedConvolution1d :
    '''
    Transpose of 1-dimensional convolution operator used for the 
    transpose-convolution operation A.T@(...)
    '''
    def __init__(self, filt) :
        self.__filt = filt
        self.__r = filt.size

    def __matmul__(self, vector) :
        r = self.__r
        n = vector.size + r - 1

        return np.asarray([np.flip(np.concatenate((np.zeros(n-r), self.__filt, np.zeros(n-r)))[i:i+n-r+1]).dot(vector) for i in np.arange(n)])

def huber_loss(x) :
    return np.sum( (1/2)*(x**2)*(np.abs(x)<=1) + (np.sign(x)*x-1/2)*(np.abs(x)>1) )
def huber_grad(x) :
    return x*(np.abs(x)<=1) + np.sign(x)*(np.abs(x)>1)


r, n, lam = 3, 20, 0.1

np.random.seed(0)
k = np.random.randn(r)
b = np.random.randn(n-r+1)
A = Convolution1d(k)
#from scipy.linalg import circulant
#A = circulant(np.concatenate((np.flip(k),np.zeros(n-r))))[r-1:,:]

x = np.zeros(n)
alpha = 0.01
for _ in range(100) :
    x = x - alpha*(A.T@(huber_grad(A@x-b))+lam*x)

print(huber_loss(A@x-b)+0.5*lam*np.linalg.norm(x)**2)

0.4587586843129764
